In [0]:
import timeit
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, \
                                    cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import plotly.tools as pt
import plotly.io as pio
import chart_studio as cs
import plotly.graph_objs as go
import seaborn as sns
sns.set('paper')

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# transaction_df = pd.read_csv('gdrive/My Drive/ieee-fraud-detection/train_transaction.csv')

In [0]:

# cat_cols = [column for column in transaction_df.columns if transaction_df[column].dtype == np.dtype('O') or transaction_df[column].value_counts().keys().shape[0] <= 2]

# dummies = pd.get_dummies(transaction_df[cat_cols].astype('category'),drop_first=True,dummy_na=True).astype('category')

# transaction_df = pd.concat([transaction_df.drop(cat_cols,axis=1), dummies],axis=1)

# null_cols = [column for column in transaction_df.columns if transaction_df[column].value_counts().keys().shape[0] == 1]

# transaction_df = transaction_df.drop(null_cols,axis=1)

# transaction_df = transaction_df.rename(columns={'isFraud_1.0':'isFraud'})

# card1 = transaction_df.card1.astype('category')

# transaction_df = pd.concat([transaction_df.drop('card1',axis=1),card1],axis=1)

In [0]:
# data_types = [transaction_df[column].dtype for column in transaction_df.columns]

In [0]:
# data_types.reverse()

In [0]:
# cat_start_idx = len(data_types) - data_types.index(np.dtype('float64'))

In [0]:
# remove_n = 589000
# drop_indices = np.random.choice(transaction_df.index,remove_n,replace=False)
# transaction_df = transaction_df.drop(drop_indices)

In [0]:
# transaction_df.to_csv('gdrive/My Drive/ieee-fraud-detection/transaction_df_compressed.csv')

In [0]:
transaction_df = pd.read_csv('./ieee-fraud-detection/transaction_df_compressed.csv')

Number of NAN per variable barplot

In [11]:
features = transaction_df.iloc[:,:cat_start_idx].isna().sum().keys()
missing = transaction_df.iloc[:,:cat_start_idx].isna().sum().values

layout = go.Layout(
autosize=True,
title='Missing Values per Feature',
yaxis=dict(
    autorange=True,
    showgrid=True,
    zeroline=True,
    dtick=0,
    gridcolor='rgb(255, 255, 255)',
    gridwidth=1,
    zerolinecolor='rgb(255, 255, 255)',
    zerolinewidth=2,
),
margin=dict(
    l=40,
    r=30,
    b=80,
    t=100,
),
paper_bgcolor='rgb(243, 243, 243)',
plot_bgcolor='rgb(243, 243, 243)',
showlegend=False
)

fig = go.Figure([go.Bar(x=features, y = missing,
                        marker_color = 'rgba(207, 114, 255, 0.5)')],layout=layout)
fig.show()

Class balance barplot

In [12]:
not_fraud_count = transaction_df['isFraud'].value_counts().values[0]
fraud_count = transaction_df['isFraud'].value_counts().values[1]

layout = go.Layout(
autosize=True,
title='Class Imbalance',
yaxis=dict(
    autorange=True,
    showgrid=True,
    zeroline=True,
    dtick=0,
    gridcolor='rgb(255, 255, 255)',
    gridwidth=1,
    zerolinecolor='rgb(255, 255, 255)',
    zerolinewidth=2,
),
margin=dict(
    l=40,
    r=30,
    b=80,
    t=100,
),
paper_bgcolor='rgb(243, 243, 243)',
plot_bgcolor='rgb(243, 243, 243)',
showlegend=False
)

fig = go.Figure([go.Bar(x=['Fraud','Not Fraud'], y = [fraud_count/transaction_df.shape[0],
                                                      not_fraud_count/transaction_df.shape[0]], 
                        marker_color = 'rgba(255, 65, 54, 0.5)')], layout=layout)
fig.show()

Train-Test Split

In [0]:
X = np.array(transaction_df.drop(['isFraud','TransactionID'],axis=1))

In [0]:
Y = np.array(transaction_df['isFraud']).reshape((-1,))

In [0]:
del transaction_df

In [0]:
x_train09, x_test09, y_train09, y_test09 = train_test_split(X,Y, test_size = 0.1, stratify = Y)
x_train05, x_test05, y_train05, y_test05 = train_test_split(X,Y, test_size = 0.5, stratify = Y)

Missingness Imputation

In [0]:
imputer = SimpleImputer().fit(x_train09)
x_train09 = imputer.transform(x_train09)
x_test09 = imputer.transform(x_test09)

scaler = StandardScaler().fit(x_train09)
x_train09 = scaler.transform(x_train09)
x_test09 = scaler.transform(x_test09)

Model Fitting for N-learn = 0.9

In [0]:
### Data capture lists: Train/Test/Cross-Validation Errors and Execution Time

logit_train_ls = []
logit_test_ls = []
logit_auc = []


logit_ridge_train_ls = []
logit_ridge_test_ls = []
logit_ridge_auc = []

logit_lasso_train_ls = []
logit_lasso_test_ls = []
logit_lasso_auc = []

rf_train_ls = []
rf_test_ls = []
rf_auc = []

svm_train_ls = []
svm_test_ls = []
svm_auc = []

logit_time = []
logit_ridge_time = []
logit_lasso_time = []
svm_time = []
rf_time = []

logit_ridge_cv_time = []
logit_lasso_cv_time = []
svm_cv_time = []



In [0]:
sample_weights = compute_sample_weight('balanced',y_train09)

param_grid_logit = {'C':np.linspace(1e-4,1,25)}


In [0]:


### Logistic Regression

logit = LogisticRegression(penalty = 'none','balanced',n_jobs=-1,
                           solver = 'lbfgs')

t_start = timeit.default_timer()
logit.fit(x_train09,y_train09,sample_weight=sample_weights)
t_end = timeit.default_timer()

train_preds = logit.predict(x_train09)
test_preds = logit.predict(x_test09)

logit_train_ls.append(1-accuracy_score(y_train09,train_preds))
logit_test_ls.append(1-accuracy_score(y_test09, test_preds))

logit_auc.append(roc_auc_score(y_test09, test_preds))

logit_time.append(t_end-t_start)


In [0]:

### Logistic Ridge CV (L2 Penalty)


ridge = LogisticRegression(penalty = 'l2', solver = 'lbfgs', n_jobs = -1, class_weight = 'balanced')

ridgeCV = GridSearchCV(ridge, param_grid_logit, cv = 10, n_jobs = -1)

t_start = timeit.default_timer()
ridgeCV.fit(x_train09, y_train09)
t_end = timeit.default_timer()

logit_ridge_cv_time.append(t_end-t_start)

ridge = ridgeCV.best_estimator_

t_start = timeit.default_timer()
ridge.fit(x_train09, y_train09,sample_weight=sample_weights)
t_end = timeit.default_timer()

logit_ridge_time.append(t_end-t_start)

train_preds = ridge.predict(x_train09)
test_preds = ridge.predict(x_test09)
probs = ridge.predict_proba(x_test09)

logit_ridge_train_ls.append(1-accuracy_score(y_train09, train_preds))
logit_ridge_test_ls.append(1-accuracy_score(y_test09, test_preds))
logit_ridge_auc.append(roc_auc_score(y_test09, probs[:,1]))




In [189]:
### Logistic Lasso CV (L1 Penalty)

lasso = LogisticRegression(penalty = 'l1', solver = 'saga', n_jobs = -1, class_weight = 'balanced')

lassoCV = GridSearchCV(lasso, param_grid_logit, cv = 10, n_jobs = -1)

t_start = timeit.default_timer()
lassoCV.fit(x_train09, y_train09)
t_end = timeit.default_timer()

logit_lasso_cv_time.append(t_end-t_start)

lasso = lassoCV.best_estimator_

t_start = timeit.default_timer()
lasso.fit(x_train09, y_train09,sample_weight=sample_weights)
t_end = timeit.default_timer()

logit_lasso_time.append(t_end-t_start)

train_preds = lasso.predict(x_train09)
test_preds = lasso.predict(x_test09)
probs = lasso.predict_proba(x_test09)

logit_lasso_train_ls.append(1-accuracy_score(y_train09, train_preds))
logit_lasso_test_ls.append(1-accuracy_score(y_test09, test_preds))
logit_lasso_auc.append(roc_auc_score(y_test09, probs[:,1]))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [0]:
abs_coefs = [abs(item) for item in lasso.coef_[0].tolist()]
cols = transaction_df.drop(['isFraud','TransactionID'],axis=1).columns

coefs = sorted(list(zip(abs_coefs,cols)),key = lambda x: x[0],reverse=True)

abs_coefs = [item[0] for item in coefs]
cols = [item[1] for item in coefs]

In [191]:
layout = go.Layout(
autosize=True,
title='Lasso Coefficient Importance (Top 20 Magnitude)',
yaxis=dict(
    autorange=True,
    showgrid=True,
    zeroline=True,
    dtick=0,
    gridcolor='rgb(255, 255, 255)',
    gridwidth=1,
    zerolinecolor='rgb(255, 255, 255)',
    zerolinewidth=2,
),
margin=dict(
    l=40,
    r=30,
    b=80,
    t=100,
),
paper_bgcolor='rgb(243, 243, 243)',
plot_bgcolor='rgb(243, 243, 243)',
showlegend=False
)

fig = go.Figure(layout=layout)

fig.add_trace(go.Bar(x = cols[:20], y = abs_coefs[:20]))

fig.show()

In [0]:
### Random Forest

rf = RandomForestClassifier(n_estimators = 300, n_jobs = -1, class_weight = 'balanced')

t_start = timeit.default_timer()
rf.fit(x_train09, y_train09,sample_weight=sample_weights)
t_end = timeit.default_timer()

rf_time.append(t_end-t_start)

train_preds = rf.predict(x_train09)
test_preds = rf.predict(x_test09)
probs = rf.predict_proba(x_test09)

rf_train_ls.append(1-accuracy_score(y_train09, train_preds))
rf_test_ls.append(1-accuracy_score(y_test09, test_preds))
rf_auc.append(roc_auc_score(y_test09, probs[:,1]))



In [129]:
rf_test_ls

[0.03937007874015752, 0.04330708661417326]

In [0]:
x_train09_regularized = x_train09[:,[item != 0 for item in lasso.coef_[0].tolist()]]
x_test09_regularized = x_test09[:,[item != 0 for item in lasso.coef_[0].tolist()]]

In [123]:
### SVM Radial Kernel

svm_param_grid = {'C':np.linspace(10**-2, 10**2, 5),
                  'gamma':np.linspace(10**-2, 10**2, 5)}

svm = SVC(kernel='rbf', probability=False, class_weight = 'balanced')

svmCV = GridSearchCV(svm, param_grid = svm_param_grid, n_jobs = -1, cv = 10)

t_start = timeit.default_timer()
svmCV.fit(x_train09_regularized, y_train09)
t_end = timeit.default_timer()

svm_cv_time.append(t_end-t_start)

svm = svmCV.best_estimator_

t_start = timeit.default_timer()
svm.fit(x_train09_regularized,y_train09,sample_weight = sample_weights)
t_end = timeit.default_timer()

svm_time.append(t_end-t_start)

train_preds = svm.predict(x_train09_regularized)
test_preds = svm.predict(x_test09_regularized)
# probs = svm.predict_proba(x_test09)

svm_train_ls.append(1-accuracy_score(y_train09,train_preds))
svm_test_ls.append(1-accuracy_score(y_test09,test_preds))
# svm_auc.append(roc_auc_score(y_test09, probs[:,1]))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



In [132]:
svm_cv_time

[187.54997595500026]

In [221]:
# print(logit_train_ls,logit_test_ls,logit_auc,logit_time, '\n')
print(logit_ridge_train_ls, logit_ridge_test_ls, logit_ridge_auc, logit_ridge_time, logit_ridge_cv_time)

[0.03237095363079612] [0.03937007874015752] [0.5106707317073171] [1.1066957189996174] [114.9037790350003]


In [222]:
ridgeCV.best_index_

0

In [223]:
ridgeCV.cv_results_

{'mean_fit_time': array([1.798084  , 1.78649299, 1.81619313, 1.80602582, 1.78484178,
        1.78639576, 1.81837487, 1.78740203, 1.78685431, 1.78553035,
        1.78604879, 1.80601938, 1.79574161, 1.77505424, 1.80631795,
        1.78638945, 1.81533999, 1.78532393, 1.79518249, 1.79558299,
        1.76609035, 1.78586452, 1.80574958, 1.81569953, 1.72570395]),
 'mean_score_time': array([0.00088954, 0.00080998, 0.00079281, 0.00076816, 0.00080695,
        0.00086915, 0.00083592, 0.00088634, 0.00085218, 0.00086029,
        0.00085008, 0.00083699, 0.00082502, 0.00084064, 0.00086975,
        0.00081329, 0.00073125, 0.00083914, 0.00085938, 0.00083375,
        0.00084088, 0.00087144, 0.00086334, 0.00086343, 0.00083773]),
 'mean_test_score': array([0.96806649, 0.96806649, 0.96806649, 0.96806649, 0.96806649,
        0.96806649, 0.96806649, 0.96806649, 0.96806649, 0.96806649,
        0.96806649, 0.96806649, 0.96806649, 0.96806649, 0.96806649,
        0.96806649, 0.96806649, 0.96806649, 0.96806649, 0

In [275]:
coefs = []
for alpha in np.linspace(1e-6,1e+3,100):
  ridge = LogisticRegression(penalty = 'l2', solver = 'saga', n_jobs = -1, class_weight = sample_weights, C = alpha)
  ridge.fit(x_train09, y_train09,sample_weight=sample_weights)

  coefs.append(np.abs(ridge.coef_))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:865: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:865: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:865: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:33

In [0]:
betas = [array.reshape(-1,).tolist() for array in coefs]
betas = list(zip(*betas))

In [0]:
betas = [list(beta) for beta in betas]

In [278]:
len(betas)

535

In [279]:

traces = []

for beta in betas:
  traces.append(go.Scatter(x = np.linspace(1e-6,1e+3,100).tolist(),
                            y = beta,
                            mode = 'lines'))

layout = go.Layout(
autosize=True,
title='Feature Coefficients for Varying Lambda in Lasso',
yaxis=dict(
    autorange=True,
    showgrid=True,
    zeroline=True,
    dtick=0,
    gridcolor='rgb(255, 255, 255)',
    gridwidth=1,
    zerolinecolor='rgb(255, 255, 255)',
    zerolinewidth=2,
),
margin=dict(
    l=40,
    r=30,
    b=80,
    t=100,
),
paper_bgcolor='rgb(243, 243, 243)',
plot_bgcolor='rgb(243, 243, 243)',
showlegend=False
)

fig = go.Figure(data=traces,layout=layout)

fig.show()